# API Testing Notebook

### Imports

In [43]:
import requests
import datetime
import pytz

import os.path

from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2 import service_account

from git import Repo
import yaml
import re
import matplotlib
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
import json

### Functions

In [44]:

def section_names(headers):
    sections_url = f"https://api.todoist.com/rest/v2/sections?project_id={tokens['todoist_projid']}"
    sc = requests.get(sections_url, headers=headers)
    s = sc.json()
    section_dict = {}
    for section in s:
        section_dict[section["id"]] = section["name"]

    return section_dict

### Secrets

In [45]:
with open('secrets/secrets.json') as secrets_file:
    tokens = json.load(secrets_file)

version = 'testing'

### Sheets Dependencies

In [48]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
ss_id = tokens["sheet_id"]

creds = service_account.Credentials.from_service_account_file(
    tokens["g_token_file"], scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

### Todoist

In [70]:
message_list = []
headers = {
    'Authorization': f'Bearer {tokens["todoist1"]}',
}
section_dict = section_names(headers)

query_url = f"https://api.todoist.com/rest/v2/tasks?project_id={tokens['todoist_projid']}"
res = requests.get(query_url, headers=headers)
r = res.json()

# parent_data = sheet.values().get(spreadsheetId=ss_id, range="Sheet2!A:B").execute()
# if "values" in parent_data.keys():
#     parent_data = parent_data["values"]
#     parent_dict = {v[0]: v[1] for v in parent_data[1:]}
# else:
#     parent_dict = {}

results = {}
parents = {}
ress = {}
for result in r:
    query_url = f"https://api.todoist.com/sync/v9/items/get?item_id={result['id']}"
    res = requests.get(query_url, headers=headers)
    results[result["id"]] = result
    ress[result["id"]] = res

    content_text = result["content"]
    if content_text[:2] == "* ":
        content_text = content_text[2:]
    # parent_dict[result["id"]] = content_text

for result, res in zip(results.values(), ress.values()):
    item = res.json()["item"]
    description = item["description"]
    section_id = item["section_id"]
    mtime = result["created_at"]
    item_parents = {}

    current_item = result
    while True:
        if current_item["parent_id"] is None:
            break
        else:
            current_item = results[current_item["parent_id"]]
            current_item_text = current_item["content"]
            if current_item_text[:2] == "* ":
                current_item_text = current_item_text[2:]
            item_parents[current_item["id"]] = current_item_text

    # if id not in parent_dict.keys():
    #     missing_tasks.append([id, result["content"]])

    dtime = datetime.datetime.strptime(mtime, "%Y-%m-%dT%H:%M:%S.%fZ")

    if result["priority"] != 1:
        prio_text = '<span style="color: #ba0202;">Priority</span>'
    else:
        prio_text = 'Open'

    if len(description) != 0:
        desc_text = f'<br><span class="desc">{description}</span>'
    else:
        desc_text = ''

    if all([section_id != 0, section_id is not None]):
        if len(item_parents) != 0:
            hierarchy_list = [v for v in item_parents.values()]
            hierarchy_list.reverse()
            hierarchy_flavor_text = '/' + '/'.join(hierarchy_list)
        else:
            hierarchy_flavor_text = ''

        if result["due"] is not None:
            at_datetime = datetime.datetime.strptime(result["due"]["datetime"], "%Y-%m-%dT%H:%M:%S")
            at_flavor_text = f" on {at_datetime.strftime('%m/%d')} at {at_datetime.strftime('%I:%M %p')}"
        else:
            at_flavor_text = ''

        sect_id_text = f' under <b>{section_dict[section_id]}{hierarchy_flavor_text}{at_flavor_text}</b>'
    else:
        sect_id_text = ''

    m = f"{prio_text} Todoist task{sect_id_text}||{result['content']}{desc_text}"
    print(m)

    if result['content'][0] != "*":
        message_list.append(
                {"time": dtime, "message": m, "via": "Todoist", "id": f'todoist_{result["id"]}', "mtime": mtime,
                 "version": version, "priority": result["priority"]})

# sheet.values().append(
#         spreadsheetId=ss_id, range=f"Sheet2!A:B", valueInputOption="RAW",
#         body={"values": missing_tasks}).execute()

Open Todoist task under <b>TDoT GI</b>||* Meetings
Open Todoist task under <b>TDoT GI/Meetings on 06/03 at 09:00 AM</b>||TDoT Weekly Training/Development meeting
Open Todoist task under <b>BCBS</b>||* Meetings
Open Todoist task under <b>TDoT GI</b>||Short implement simplified STRATUM for training purposes
Open Todoist task under <b>BCBS</b>||* HSC
Open Todoist task under <b>BCBS/HSC</b>||Implement bnft_mbr<br><span class="desc">[http://shareatwork/c/DMGI/Shared%20Documents/DW%20Website/Info_Tips/Identifying_LOB.pdf](http://shareatwork/c/DMGI/Shared%20Documents/DW%20Website/Info_Tips/Identifying_LOB.pdf)
fi = fully insured, aso = self funded
HDP + FSA?</span>
Open Todoist task under <b>BCBS/HSC</b>||Implement med_util
Open Todoist task under <b>BCBS/HSC</b>||Start building proto-data pipeline to model HSC
Open Todoist task under <b>daxtracker.</b>||* Features
Open Todoist task under <b>daxtracker./Features</b>||Implement task hierarchies on flavor text
Open Todoist task under <b>daxtrac

In [69]:
event_dict = {}
message_list = []

headers = {
    'Authorization': f'Bearer {tokens["todoist1"]}',
}
section_dict = section_names(headers)
query_url = f"https://api.todoist.com/sync/v9/completed/get_all?project_id={tokens['todoist_projid']}"
res = requests.get(query_url, headers=headers)
r = res.json()["items"]
for result in r:
    if f"todoist_{result['id']}" not in event_dict.keys():
        query_url = f"https://api.todoist.com/sync/v9/items/get?item_id={result['task_id']}"
        res = requests.get(query_url, headers=headers)
        print(res.json()["item"])
        try:
            item = res.json()["item"]
        except KeyError:
            description = ''
            section_id = None
        else:
            description = item["description"]
            section_id = item["section_id"]

        if len(description) != 0:
            desc_text = f'<br><span class="desc">{description}</span>'
        else:
            desc_text = ''

        item_parents = {}
        # print(result)

        # current_item = result
        # while True:
        #     if current_item["parent_id"] is None:
        #         break
        #     else:
        #         current_item = results[current_item["parent_id"]]
        #         current_item_text = current_item["content"]
        #         if current_item_text[:2] == "* ":
        #             current_item_text = current_item_text[2:]
        #         item_parents[result['task_id']] = current_item_text

        # print(result)
        if section_id is not None:
            if res.json()["item"]["due"] is not None:
                at_datetime = datetime.datetime.strptime(res.json()["item"]["due"]["datetime"], "%Y-%m-%dT%H:%M:%S")
                at_flavor_text = f" on {at_datetime.strftime('%m/%d')} at {at_datetime.strftime('%H:%M')}"
            else:
                at_flavor_text = ''

            sect_id_text = f' under <b>{section_dict[section_id]}{at_flavor_text}</b>'
            # sect_id_text = f' under <b>{section_dict[section_id]}</b>'
        else:
            sect_id_text = ''

        m = f"Closed Todoist task{sect_id_text}||{result['content']}{desc_text}"
    else:
        m = event_dict[f"todoist_{result['id']}"][1]

    # Thanks to Jon Betts on https://stackoverflow.com/questions/23394608/python-regex-fails-to-identify-markdown-links
    name_regex = "[^]]+"
    url_regex = "http[s]?://[^)]+"
    link_regex = '\[({0})]\(\s*({1})\s*\)'.format(name_regex, url_regex)

    # Thanks to NPE & kubanczyk on https://stackoverflow.com/questions/7191209/re-sub-replace-with-matched-content
    m = re.sub(link_regex, r'<a href="\2">\1</a>', m)

    mtime = result["completed_at"]
    dtime = datetime.datetime.strptime(mtime, "%Y-%m-%dT%H:%M:%S.%fZ")
    message_list.append(
        {"time": dtime, "message": m, "via": "Todoist", "id": f'todoist_{result["id"]}', "mtime": mtime,
         "version": version})

{'added_at': '2023-05-25T13:37:39.099679Z', 'added_by_uid': '34598745', 'assigned_by_uid': None, 'checked': True, 'child_order': 2, 'collapsed': False, 'completed_at': '2023-06-02T13:52:39.909703Z', 'content': 'Make **TDoT GI Technical Milestones document**', 'description': '', 'due': None, 'id': '6909683084', 'is_deleted': False, 'labels': [], 'parent_id': None, 'priority': 1, 'project_id': '2273569911', 'responsible_uid': None, 'section_id': '123494547', 'sync_id': None, 'user_id': '34598745'}
{'added_at': '2023-06-01T18:28:48.379110Z', 'added_by_uid': '34598745', 'assigned_by_uid': None, 'checked': True, 'child_order': 1, 'collapsed': False, 'completed_at': '2023-06-01T18:28:52.075798Z', 'content': 'Weekly DSCoE Training/Working Session', 'description': '', 'due': None, 'id': '6930020229', 'is_deleted': False, 'labels': [], 'parent_id': '6920222142', 'priority': 1, 'project_id': '2273569911', 'responsible_uid': None, 'section_id': '110111394', 'sync_id': None, 'user_id': '34598745'}